In [6]:
# CELL 1: Project directory setup
# ----------------------------------------------
# This cell automatically detects the project root
# and initializes the directories for raw and processed data.

from pathlib import Path
import os

# Detect project root:
# If notebook is inside 'notebooks/', use its parent as project root.
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name.lower() in ["notebook", "notebooks"]:
    PROJECT_ROOT = PROJECT_ROOT.parent
elif not (PROJECT_ROOT / "data").exists():
    # If current directory isn't the project root, go one level up
    PROJECT_ROOT = PROJECT_ROOT.parent

# Define folder paths
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

# Create folders if they do not exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT   :", PROJECT_ROOT)
print("DATA_RAW       :", DATA_RAW)
print("DATA_PROCESSED :", DATA_PROCESSED)

PROJECT_ROOT   : /Users/ibrahimgozlukaya/Desktop/DSA210
DATA_RAW       : /Users/ibrahimgozlukaya/Desktop/DSA210/data/raw
DATA_PROCESSED : /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed


In [9]:
# CELL 3: Download merged monthly social-media share panel
# ---------------------------------------------------------
# This loops through all European countries, downloads their Statcounter
# monthly CSV, parses it, adds country info, and concatenates everything
# into a single panel.
# The final file is saved under data/processed/

import io
import requests
import pandas as pd

all_frames = []

for geo, info in tries.items():
    slug = info["slug"]
    name_in_url = info["name_in_url"]

    # Build the country-specific Statcounter URL
    url = build_url(geo, slug, name_in_url)
    print(f"Downloading: {geo} → {slug}")

    # Download CSV content
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
    except Exception as e:
        print(f"❌ Connection error for {geo}: {e}")
        continue

    # Parse CSV
    try:
        df = pd.read_csv(io.StringIO(r.text))
        df["geo"] = geo
        df["country_name"] = name_in_url
        all_frames.append(df)
    except Exception as e:
        print(f"❌ CSV parsing failed for {geo}: {e}")

# Merge all countries into one dataframe
if all_frames:
    panel = pd.concat(all_frames, ignore_index=True)
    out_path = DATA_PROCESSED / "social_media_countries_monthly.csv"
    panel.to_csv(out_path, index=False)

    print("\n✅ Merged Statcounter file saved:", out_path)
    print("Total rows:", len(panel))
else:
    print("❌ No Statcounter data downloaded.")

Downloading: AL → albania
Downloading: AT → austria
Downloading: BA → bosnia-and-herzegovina
Downloading: BE → belgium
Downloading: BG → bulgaria
Downloading: CH → switzerland
Downloading: CY → cyprus
Downloading: CZ → czech-republic
Downloading: DE → germany
Downloading: DK → denmark
Downloading: EE → estonia
Downloading: EL → greece
Downloading: ES → spain
Downloading: FI → finland
Downloading: FR → france
Downloading: HR → croatia
Downloading: HU → hungary
Downloading: IE → ireland
Downloading: IS → iceland
Downloading: IT → italy
Downloading: LT → lithuania
Downloading: LU → luxembourg
Downloading: LV → latvia
Downloading: ME → montenegro
Downloading: MK → north-macedonia
Downloading: MT → malta
Downloading: NL → netherlands
Downloading: NO → norway
Downloading: PL → poland
Downloading: PT → portugal
Downloading: RO → romania
Downloading: RS → serbia
Downloading: SE → sweden
Downloading: SI → slovenia
Downloading: SK → slovakia
Downloading: TR → turkey
Downloading: UK → united-king

In [10]:
# CELL 4: Define RAW directory for Eurostat & OECD files
# -------------------------------------------------------
# All externally downloaded CSV files (Eurostat, OECD)
# are expected to be stored under data/raw/.

import pandas as pd
import numpy as np
import os

DATA_RAW = DATA_RAW  # already defined in CELL 1
DATA_PROCESSED = DATA_PROCESSED  # already defined in CELL 1

print("RAW directory      :", DATA_RAW)
print("PROCESSED directory:", DATA_PROCESSED)

RAW directory      : /Users/ibrahimgozlukaya/Desktop/DSA210/data/raw
PROCESSED directory: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed


In [11]:
# CELL 5: Eurostat tin00127 — annual social media participation
# -------------------------------------------------------------
# Source: tin00127_linear_2_0.csv
# We use:
#   - geo          : country code (ISO-2 style)
#   - TIME_PERIOD  : year (YYYY)
#   - OBS_VALUE    : share of individuals using social networks (% of individuals)

tin_path = DATA_RAW / "tin00127_linear_2_0.csv"
tin_raw = pd.read_csv(tin_path)

print("tin00127 raw shape:", tin_raw.shape)
display(tin_raw.head())
# Clean and keep only the relevant columns
tin = tin_raw.rename(columns={
    "TIME_PERIOD": "year",
    "OBS_VALUE": "sm_participation",
})

# Ensure numeric year
tin["year"] = tin["year"].astype(int)

tin = tin[["geo", "year", "sm_participation"]].copy()

print("Clean tin shape:", tin.shape)
display(tin.head())

tin00127 raw shape: (424, 21)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,indic_is,Information society indicator,ind_type,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2018,NaN,48.33,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2019,NaN,52.13,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2020,NaN,54.80,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2021,NaN,60.77,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2022,NaN,64.07,NaN,NaN,NaN,NaN,NaN


Clean tin shape: (424, 3)


,geo,year,sm_participation
0,AL,2018,48.33
1,AL,2019,52.13
2,AL,2020,54.80
3,AL,2021,60.77
4,AL,2022,64.07


In [15]:
# CELL 6: Eurostat HICP — monthly index to inflation measures
# -----------------------------------------------------------
# Source: prc_hicp_midx__custom_19133241_linear.csv
# Important columns:
#   - geo        : country name (e.g. "Austria", "Belgium", "Türkiye")
#   - TIME_PERIOD: monthly date string "YYYY-MM"
#   - OBS_VALUE  : HICP index (2015 = 100)

hicp_path = DATA_RAW / "prc_hicp_midx__custom_19134087_linear.csv"
hicp_raw = pd.read_csv(hicp_path)

print("HICP raw shape:", hicp_raw.shape)
display(hicp_raw.head())
# Rename to clearer names
hicp = hicp_raw.rename(columns={
    "geo": "geo_name",
    "TIME_PERIOD": "date",
    "OBS_VALUE": "hicp_index",
})

# Parse date and extract year/month
hicp["date"] = pd.to_datetime(hicp["date"])
hicp["year"] = hicp["date"].dt.year
hicp["month"] = hicp["date"].dt.month

print("Example country names in HICP:", sorted(hicp["geo_name"].unique())[:10])
# Map country names (in HICP) to Eurostat geo codes (used in tin00127 and unemployment)
name_to_geo = {
    "Albania": "AL",
    "Austria": "AT",
    "Belgium": "BE",
    "Bulgaria": "BG",
    "Croatia": "HR",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Denmark": "DK",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Germany": "DE",
    "Greece": "EL",
    "Hungary": "HU",
    "Iceland": "IS",
    "Ireland": "IE",
    "Italy": "IT",
    "Latvia": "LV",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Malta": "MT",
    "Montenegro": "ME",
    "Netherlands": "NL",
    "North Macedonia": "MK",
    "Norway": "NO",
    "Poland": "PL",
    "Portugal": "PT",
    "Romania": "RO",
    "Serbia": "RS",
    "Slovakia": "SK",
    "Slovenia": "SI",
    "Spain": "ES",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Türkiye": "TR",
    "United Kingdom": "UK",
    # you can extend this dict if needed
}

hicp["geo"] = hicp["geo_name"].map(name_to_geo)

# Keep only rows where we have a valid geo code
hicp = hicp[~hicp["geo"].isna()].copy()

print("Clean HICP shape (with geo codes):", hicp.shape)
display(hicp[["geo_name", "geo", "date", "hicp_index"]].head())
# (1) Monthly year-on-year inflation (for H3 monthly panel)
# --------------------------------------------------------
# Year-on-year inflation: percentage change compared to the same month one year earlier.

hicp = hicp.sort_values(["geo", "date"])
hicp["hicp_index_lag12"] = hicp.groupby("geo")["hicp_index"].shift(12)
hicp["inflation_yoy"] = (hicp["hicp_index"] / hicp["hicp_index_lag12"] - 1) * 100

hicp_monthly = hicp[["geo", "year", "month", "inflation_yoy"]].copy()

print("HICP monthly (yoy inflation) shape:", hicp_monthly.shape)
display(hicp_monthly.head())
# (2) Annual inflation based on December-to-December changes (for H1/H2 annual panel)
# -----------------------------------------------------------------------------------
# Annual inflation = (December index this year / December index last year - 1) * 100

december = hicp[hicp["month"] == 12].copy()
december = december.sort_values(["geo", "year"])
december["inflation"] = december.groupby("geo")["hicp_index"].pct_change() * 100

inflation_annual = december[["geo", "year", "inflation"]].copy()

print("Annual inflation shape:", inflation_annual.shape)
display(inflation_annual.head())

HICP raw shape: (8636, 10)


,DATAFLOW,LAST UPDATE,freq,unit,coicop,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-01,105.58,NaN,NaN
1,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-02,105.90,NaN,NaN
2,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-03,106.86,NaN,NaN
3,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-04,107.15,NaN,NaN
4,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-05,107.72,NaN,NaN


Example country names in HICP: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)', 'Euro area - 19 countries  (2015-2022)']
Clean HICP shape (with geo codes): (7003, 13)


,geo_name,geo,date,hicp_index
0,Austria,AT,2008-01-01,105.58
1,Austria,AT,2008-02-01,105.90
2,Austria,AT,2008-03-01,106.86
3,Austria,AT,2008-04-01,107.15
4,Austria,AT,2008-05-01,107.72


HICP monthly (yoy inflation) shape: (7003, 4)


,geo,year,month,inflation_yoy
0,AT,2008,1,NaN
1,AT,2008,2,NaN
2,AT,2008,3,NaN
3,AT,2008,4,NaN
4,AT,2008,5,NaN


Annual inflation shape: (556, 3)


,geo,year,inflation
11,AT,2008,NaN
23,AT,2009,1.051750
35,AT,2010,2.173713
47,AT,2011,3.398540
59,AT,2012,2.920663


In [16]:
# CELL 7: Eurostat unemployment — monthly rates
# ---------------------------------------------
# Source: une_rt_m_linear_2_0.csv
# We select:
#   - s_adj = "SA"       : seasonally adjusted
#   - age   = "TOTAL"    : total working-age population
#   - sex   = "T"        : both sexes
#   - unit  = "PC_ACT"   : percent of active population
# Then compute monthly series and annual averages.

unemp_path = DATA_RAW / "une_rt_m_linear_2_0.csv"
unemp_raw = pd.read_csv(unemp_path)

print("Unemployment raw shape:", unemp_raw.shape)
display(unemp_raw.head())
# Filter to the standard unemployment rate definition
unemp_filtered = unemp_raw[
    (unemp_raw["s_adj"] == "SA") &
    (unemp_raw["age"] == "TOTAL") &
    (unemp_raw["sex"] == "T") &
    (unemp_raw["unit"] == "PC_ACT")
].copy()

print("Filtered unemployment shape:", unemp_filtered.shape)
display(unemp_filtered.head())
# Parse date and extract year/month
unemp_filtered = unemp_filtered.rename(columns={
    "TIME_PERIOD": "date",
    "OBS_VALUE": "unemployment",
})

unemp_filtered["date"] = pd.to_datetime(unemp_filtered["date"])
unemp_filtered["year"] = unemp_filtered["date"].dt.year
unemp_filtered["month"] = unemp_filtered["date"].dt.month

unemp_monthly = unemp_filtered[["geo", "year", "month", "unemployment"]].copy()

print("Unemployment monthly shape:", unemp_monthly.shape)
display(unemp_monthly.head())
# Annual average unemployment rate
unemp_annual = (
    unemp_monthly
    .groupby(["geo", "year"], as_index=False)["unemployment"]
    .mean()
)

print("Unemployment annual shape:", unemp_annual.shape)
display(unemp_annual.head())

Unemployment raw shape: (727293, 23)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,s_adj,Seasonal adjustment,age,Age class,unit,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-01,NaN,5.5,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-02,NaN,5.5,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-03,NaN,5.4,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-04,NaN,5.6,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-05,NaN,5.6,NaN,NaN,NaN,NaN,NaN


Filtered unemployment shape: (14263, 23)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,s_adj,Seasonal adjustment,age,Age class,unit,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
274526,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-01,NaN,4.0,NaN,NaN,NaN,NaN,NaN
274527,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-02,NaN,3.9,NaN,NaN,NaN,NaN,NaN
274528,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-03,NaN,3.9,NaN,NaN,NaN,NaN,NaN
274529,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-04,NaN,4.0,NaN,NaN,NaN,NaN,NaN
274530,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-05,NaN,4.1,NaN,NaN,NaN,NaN,NaN


Unemployment monthly shape: (14263, 4)


,geo,year,month,unemployment
274526,AT,1995,1,4.0
274527,AT,1995,2,3.9
274528,AT,1995,3,3.9
274529,AT,1995,4,4.0
274530,AT,1995,5,4.1


Unemployment annual shape: (1199, 3)


,geo,year,unemployment
0,AT,1995,4.241667
1,AT,1996,4.716667
2,AT,1997,4.758333
3,AT,1998,4.708333
4,AT,1999,4.141667


In [17]:
# CELL 8: OECD CCI export — wide to long, monthly & annual CCI
# -------------------------------------------------------------
# Source: export-2025-11-26T13_03_48.204Z.csv
# This file is a wide table where:
#   - "Category" column contains dates ("2010-07-01 00:00:00")
#   - Each other column is a country or region
#   - Values use a comma as decimal separator and semicolon as delimiter

cci_path = DATA_RAW / "export-2025-11-26T13_03_48.204Z.csv"
cci_wide = pd.read_csv(
    cci_path,
    sep=";",        # semicolon-separated
    skiprows=2,     # skip the first two metadata rows
    decimal=",",    # use comma as decimal separator
)

print("CCI wide shape:", cci_wide.shape)
display(cci_wide.head())
# Select only European country columns that we can map to Eurostat geo codes
cci_name_to_geo = {
    "Austria": "AT",
    "Belgium": "BE",
    "Czechia": "CZ",
    "Denmark": "DK",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Germany": "DE",
    "Greece": "EL",
    "Hungary": "HU",
    "Iceland": "IS",
    "Ireland": "IE",
    "Italy": "IT",
    "Latvia": "LV",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Netherlands": "NL",
    "Norway": "NO",
    "Poland": "PL",
    "Portugal": "PT",
    "Slovak Rep.": "SK",
    "Slovenia": "SI",
    "Spain": "ES",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Türkiye": "TR",
    "United Kingdom": "UK",
}

europe_cols = ["Category"] + [name for name in cci_name_to_geo.keys() if name in cci_wide.columns]
cci_europe = cci_wide[europe_cols].copy()

print("CCI Europe subset shape:", cci_europe.shape)
display(cci_europe.head())# Convert from wide (one column per country) to long (one row per country-month)
cci_long = cci_europe.melt(
    id_vars="Category",
    var_name="country_name",
    value_name="cci",
)

# Map country names to Eurostat geo codes
cci_long["geo"] = cci_long["country_name"].map(cci_name_to_geo)
cci_long = cci_long[~cci_long["geo"].isna()].copy()

# Parse date and extract year/month
cci_long["date"] = pd.to_datetime(cci_long["Category"])
cci_long["year"] = cci_long["date"].dt.year
cci_long["month"] = cci_long["date"].dt.month

cci_monthly = cci_long[["geo", "year", "month", "cci"]].dropna()

print("CCI monthly shape:", cci_monthly.shape)
display(cci_monthly.head())
# Annual average CCI
cci_annual = (
    cci_monthly
    .groupby(["geo", "year"], as_index=False)["cci"]
    .mean()
)

print("CCI annual shape:", cci_annual.shape)
display(cci_annual.head())

CCI wide shape: (184, 50)


,Category,Major five Asia economies,Australia,Austria,Belgium,Chile,Colombia,Costa Rica,Czechia,Denmark,...,European Union (since 2020),OECD,OECD excluding the euro area,OECD Europe,Brazil,China,India,Indonesia,Russia,South Africa
0,2010-07-01 00:00:00,100.16980,101.1949,100.2935,100.5633,101.5922,103.7792,102.9870,99.95223,101.1339,...,98.71791,99.25383,99.17513,99.47222,102.9224,99.73689,NaN,99.68826,101.2028,102.5821
1,2010-08-01 00:00:00,99.98219,101.5486,100.6558,100.8198,102.0630,104.2988,102.6625,99.65169,101.1359,...,99.14233,99.27330,99.09669,99.64468,103.0428,99.46487,NaN,99.80018,100.9577,102.6125
2,2010-09-01 00:00:00,99.70647,101.6113,100.8824,100.9553,102.6991,104.2489,102.3753,99.30165,101.1473,...,99.41579,99.29877,99.08424,99.73608,103.1625,99.05468,NaN,99.81923,100.8880,102.6349
3,2010-10-01 00:00:00,99.49446,101.5537,100.8755,101.0141,103.0448,103.7127,102.1615,98.98066,101.1613,...,99.52643,99.33994,99.15450,99.73548,103.2801,98.69066,NaN,99.87341,100.9000,102.6183
4,2010-11-01 00:00:00,99.32086,101.3013,100.6785,100.9837,102.9526,102.9605,102.0852,98.86246,101.1617,...,99.55367,99.41690,99.31138,99.70140,103.4245,98.33498,NaN,100.00740,100.8633,102.5609


CCI Europe subset shape: (184, 26)


,Category,Austria,Belgium,Czechia,Denmark,Estonia,Finland,France,Germany,Greece,...,Netherlands,Poland,Portugal,Slovak Rep.,Slovenia,Spain,Sweden,Switzerland,Türkiye,United Kingdom
0,2010-07-01 00:00:00,100.2935,100.5633,99.95223,101.1339,100.1657,101.9947,98.55846,100.7156,96.30705,...,100.0866,100.13370,97.70040,99.98700,98.21307,95.97453,102.1055,102.0776,99.98065,99.27699
1,2010-08-01 00:00:00,100.6558,100.8198,99.65169,101.1359,100.3743,102.3227,98.78314,101.2009,96.39933,...,100.1937,99.90450,97.67054,99.72249,98.07329,96.56970,102.1560,102.0857,100.15990,99.08302
2,2010-09-01 00:00:00,100.8824,100.9553,99.30165,101.1473,100.4359,102.4197,98.98823,101.5459,96.24638,...,100.2077,99.64635,97.25994,99.48306,98.13566,97.02423,102.1740,101.9900,100.49300,98.73116
3,2010-10-01 00:00:00,100.8755,101.0141,98.98066,101.1613,100.3462,102.1022,99.23347,101.7402,96.01804,...,100.2202,99.46078,96.24286,99.31108,98.30521,97.09800,102.0650,101.8351,100.71870,98.41450
4,2010-11-01 00:00:00,100.6785,100.9837,98.86246,101.1617,100.4737,101.5863,99.38045,101.8713,95.75428,...,100.2195,99.35139,95.59222,99.37595,98.26225,96.90553,101.9832,101.6675,101.00500,98.11409


CCI monthly shape: (4598, 4)


,geo,year,month,cci
0,AT,2010,7,100.2935
1,AT,2010,8,100.6558
2,AT,2010,9,100.8824
3,AT,2010,10,100.8755
4,AT,2010,11,100.6785


CCI annual shape: (400, 3)


,geo,year,cci
0,AT,2010,100.650433
1,AT,2011,99.653852
2,AT,2012,99.095582
3,AT,2013,99.597598
4,AT,2014,99.524171


In [18]:
# CELL 9: Statcounter merged monthly file — platform shares + ent/prof
# --------------------------------------------------------------------
# Source: data/processed/social_media_countries_monthly.csv
# This file was created in earlier cells by downloading
# monthly social-media market shares for each country.

sc_path = DATA_PROCESSED / "social_media_countries_monthly.csv"
sc_raw = pd.read_csv(sc_path)

print("Statcounter raw shape:", sc_raw.shape)
display(sc_raw.head())
# Identify the date column (usually "Month" or "Date")
date_col_candidates = [
    c for c in sc_raw.columns
    if c.lower().startswith("date") or c.lower().startswith("month")
]
print("Date column candidates:", date_col_candidates)

date_col = date_col_candidates[0]
print("Using date column:", date_col)

# Parse date and extract year/month
sc_raw["date"] = pd.to_datetime(sc_raw[date_col])
sc_raw["year"] = sc_raw["date"].dt.year
sc_raw["month"] = sc_raw["date"].dt.month
# Identify platform columns
candidate_platforms = ["Facebook", "Instagram", "YouTube", "TikTok", "LinkedIn"]
platform_cols = [c for c in candidate_platforms if c in sc_raw.columns]

print("Platform columns found:", platform_cols)

# Keep only geo, year, month, and platform shares
sc_monthly = sc_raw[["geo", "year", "month"] + platform_cols].copy()

# Compute entertainment vs professional shares
ent_cols = [c for c in ["Facebook", "Instagram", "YouTube", "TikTok"] if c in sc_monthly.columns]
prof_cols = [c for c in ["LinkedIn"] if c in sc_monthly.columns]

print("Entertainment platforms:", ent_cols)
print("Professional platforms:", prof_cols)

if ent_cols:
    sc_monthly["ent_share"] = sc_monthly[ent_cols].sum(axis=1)
else:
    sc_monthly["ent_share"] = np.nan

if prof_cols:
    sc_monthly["prof_share"] = sc_monthly[prof_cols].sum(axis=1)
else:
    sc_monthly["prof_share"] = np.nan

sc_monthly["ent_minus_prof"] = sc_monthly["ent_share"] - sc_monthly["prof_share"]

print("Clean Statcounter monthly shape:", sc_monthly.shape)
display(sc_monthly.head())


Statcounter raw shape: (6878, 31)


,Date,Facebook,Twitter,YouTube,Pinterest,Instagram,StumbleUpon,Tumblr,reddit,LinkedIn,...,Odnoklassniki,orkut,Fark,MySpace,Unnamed: 1,Tuenti,Netlog,FriendFeed,Hi5,Hyves
0,2010-10,86.40,0.50,11.36,0.0,0.0,1.15,0.0,0.08,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-11,87.67,0.43,10.63,0.0,0.0,0.83,0.0,0.07,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-12,87.53,0.37,10.80,0.0,0.0,0.89,0.0,0.06,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01,90.20,0.26,8.28,0.0,0.0,0.94,0.0,0.06,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-02,88.35,0.37,9.84,0.0,0.0,1.14,0.0,0.04,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Date column candidates: ['Date']
Using date column: Date
Platform columns found: ['Facebook', 'Instagram', 'YouTube', 'LinkedIn']
Entertainment platforms: ['Facebook', 'Instagram', 'YouTube']
Professional platforms: ['LinkedIn']
Clean Statcounter monthly shape: (6878, 10)


,geo,year,month,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2010,10,86.40,0.0,11.36,0.03,97.76,0.03,97.73
1,AL,2010,11,87.67,0.0,10.63,0.03,98.30,0.03,98.27
2,AL,2010,12,87.53,0.0,10.80,0.04,98.33,0.04,98.29
3,AL,2011,1,90.20,0.0,8.28,0.05,98.48,0.05,98.43
4,AL,2011,2,88.35,0.0,9.84,0.05,98.19,0.05,98.14


In [19]:
# CELL 10: Build monthly H3 panel
# -------------------------------
# This panel combines:
#   - Statcounter monthly platform shares (ent/prof)
#   - Monthly inflation (HICP, yoy)
#   - Monthly unemployment
#   - Monthly CCI
# It is used to test H3 at the monthly level.

panel_monthly_h3 = sc_monthly.copy()

panel_monthly_h3 = (
    panel_monthly_h3
    .merge(hicp_monthly, on=["geo", "year", "month"], how="left")
    .merge(unemp_monthly, on=["geo", "year", "month"], how="left")
    .merge(cci_monthly, on=["geo", "year", "month"], how="left")
)

print("Monthly H3 panel shape:", panel_monthly_h3.shape)
display(panel_monthly_h3.head())
# Save monthly H3 panel
out_monthly_h3 = DATA_PROCESSED / "panel_monthly_h3.csv"
panel_monthly_h3.to_csv(out_monthly_h3, index=False)

print("✅ Monthly H3 panel saved to:", out_monthly_h3)

Monthly H3 panel shape: (6878, 13)


,geo,year,month,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof,inflation_yoy,unemployment,cci
0,AL,2010,10,86.40,0.0,11.36,0.03,97.76,0.03,97.73,NaN,NaN,NaN
1,AL,2010,11,87.67,0.0,10.63,0.03,98.30,0.03,98.27,NaN,NaN,NaN
2,AL,2010,12,87.53,0.0,10.80,0.04,98.33,0.04,98.29,NaN,NaN,NaN
3,AL,2011,1,90.20,0.0,8.28,0.05,98.48,0.05,98.43,NaN,NaN,NaN
4,AL,2011,2,88.35,0.0,9.84,0.05,98.19,0.05,98.14,NaN,NaN,NaN


✅ Monthly H3 panel saved to: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_monthly_h3.csv


In [20]:
# CELL 11: Statcounter annual summary
# -----------------------------------
# We aggregate Statcounter platform shares to the annual level
# for use in the annual panel (H1/H2). We keep:
#   - average platform shares per year
#   - average ent_share, prof_share, ent_minus_prof per year

sc_annual = (
    sc_monthly
    .groupby(["geo", "year"], as_index=False)[platform_cols + ["ent_share", "prof_share", "ent_minus_prof"]]
    .mean()
)

print("Statcounter annual shape:", sc_annual.shape)
display(sc_annual.head())

Statcounter annual shape: (608, 9)


,geo,year,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2010,87.200000,0.0,10.930000,0.033333,98.130000,0.033333,98.096667
1,AL,2011,89.330833,0.0,7.629167,0.073333,96.960000,0.073333,96.886667
2,AL,2012,94.013333,0.0,3.511667,0.069167,97.525000,0.069167,97.455833
3,AL,2013,93.751667,0.0,1.822500,0.046667,95.574167,0.046667,95.527500
4,AL,2014,94.268333,0.0,0.569167,0.035000,94.837500,0.035000,94.802500


In [22]:
# CELL 12: Build final annual panel
# ---------------------------------
# This panel combines:
#   - Annual social-media participation (tin00127)
#   - Annual inflation (HICP December-to-December)
#   - Annual unemployment (average)
#   - Annual CCI (average)
#   - Annual Statcounter summary (platform + ent/prof shares)

panel_annual = tin.copy()

panel_annual = (
    panel_annual
    .merge(inflation_annual, on=["geo", "year"], how="left")
    .merge(unemp_annual, on=["geo", "year"], how="left")
    .merge(cci_annual, on=["geo", "year"], how="left")
    .merge(sc_annual, on=["geo", "year"], how="left")
)

print("Final annual panel shape:", panel_annual.shape)
display(panel_annual.head())
# Save final annual panel
out_annual = DATA_PROCESSED / "panel_annual.csv"
panel_annual.to_csv(out_annual, index=False)

print("✅ Annual panel saved to:", out_annual)

Final annual panel shape: (424, 13)


,geo,year,sm_participation,inflation,unemployment,cci,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2018,48.33,NaN,NaN,NaN,79.240833,3.094167,7.065000,0.298333,89.400000,0.298333,89.101667
1,AL,2019,52.13,NaN,NaN,NaN,69.003333,5.556667,4.623333,0.275833,79.183333,0.275833,78.907500
2,AL,2020,54.80,NaN,NaN,NaN,75.620000,6.143333,3.115000,0.157500,84.878333,0.157500,84.720833
3,AL,2021,60.77,NaN,NaN,NaN,91.660833,1.360000,1.035833,0.115833,94.056667,0.115833,93.940833
4,AL,2022,64.07,NaN,NaN,NaN,93.218333,1.830000,1.087500,0.140000,96.135833,0.140000,95.995833


✅ Annual panel saved to: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_annual.csv
